In [ ]:
import ROOT
from ROOT import TFile
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import copy
import pandas as pd 
import time

import seaborn as sns

import root_pandas 
from root_pandas import read_root

#
mpl.rcParams.update({'font.size': 19})
#mpl.rcParams.update({'legend.fontsize': 18})
mpl.rcParams.update({'xtick.labelsize': 18}) 
mpl.rcParams.update({'ytick.labelsize': 18}) 
mpl.rcParams.update({'text.usetex' : False})
mpl.rcParams.update({'axes.labelsize': 18}) 
mpl.rcParams.update({'legend.frameon': False}) 

In [ ]:
# Define default plot styles
plot_style_0 = {
    'histtype': 'step',
    'color': 'black',
    'linewidth': 2,
    'linestyle': '--',
    'density': True
}

plot_style_1 = {
    'histtype': 'step',
    'color': 'black',
    'linewidth': 2,
    'density': True
}

plot_style_2 = {'alpha': 0.5, 'density': True}

plot_style_3 = {
    'histtype': 'step',
    'linewidth': 2,
    'density': True,
    'alpha':0.8
}

In [ ]:
def applyCut(inputDataframe, cut, text=None):
    dataframe = inputDataframe
    nbeforecut = dataframe.shape[0]
    cutDataframe = dataframe.query(cut)
    if text:
        print (text, cutDataframe.shape[0], ' fraction kept: %2.1f'%(100.0*float(cutDataframe.shape[0])/nbeforecut))
    return cutDataframe

In [ ]:
def applyCuts(df,isMC=False):
    temp = df
    temp.eval('theta_lab = arccos(Pz/P)*180.0/3.14159',inplace=True)
    #temp.eval('inelasticity = Nu/5.014', inplace=True)
    temp.eval('E = sqrt(P*P+0.1396*0.1396)', inplace=True)
    if(isMC):
        print ('MC')
    else:
        print ('Data')
        temp.eval('pass_reco=1',inplace=True)

    #temp = applyCut(temp, 'pass_reco==0 | inelasticity<0.85','inelasticity < 0.85')
    temp = applyCut(temp, 'pass_reco==0 | (theta_lab<120 and theta_lab>10)', '10<theta_lab<120')
    temp = applyCut(temp, 'pass_reco==0 | (P>0.200)', 'P>0.200')
    
    if(not (isMC)):
        temp = applyCut(temp, 'pass_reco==0 | abs(deltaZ)<3', 'abs(deltaZ)<3 [cm]')
        #temp = applyCut(temp, 'pass_reco==0 | abs(YC)<1.4', 'abs(YC)<1.4 [cm]')

    temp = applyCut(temp,'pass_reco==0 |(pid==211)| (pid==-211 & theta_lab>30)| (pid==-211 & theta_lab<30 & P>0.600)','(pid==211)| (pid==-211 & theta_lab>30)| (pid==-211 & theta_lab<30 & P>0.600)')
    print('################\n')
    return temp

In [ ]:
path = '/home/miguel/EG2_DATA/andres/'
mc_hadron = {}
mc_hadron['D'] = root_pandas.read_root(path+'D_weighted.root', "ntuple_sim")
mc_hadron['Pb'] = root_pandas.read_root(path+'Pb_weighted.root', "ntuple_sim")


## Select targets

In [ ]:
mc_hadron['D'] = applyCut(mc_hadron['D'], 'pass_reco==0 | TargType==1','TargetType==1')
mc_hadron['Pb'] = applyCut(mc_hadron['Pb'], 'pass_reco==0 | TargType==2','TargetType==2')


## Apply cuts

In [ ]:
mc_hadron['D'] = applyCuts(mc_hadron['D'],isMC=True)
mc_hadron['Pb'] = applyCuts(mc_hadron['Pb'],isMC=True)

In [ ]:
fig = plt.figure( figsize=(8, 6))

#query = 'pass_reco==1'
x_bins=np.logspace(np.log10(0.1), np.log10(1.0),10)
#sns.regplot(x=mc_hadron.query(query)['z'],y=mc_hadron.query(query)['Pt2'],x_bins=x_bins,fit_reg=None)

query = 'pass_truth==1 and mc_pid==211'
sns.regplot(x=mc_hadron['D'].query(query)['mc_Zh'],y=mc_hadron['D'].query(query)['mc_Pt2'],label='D',x_bins=x_bins,fit_reg=None)
sns.regplot(x=mc_hadron['Pb'].query(query)['mc_Zh'],y=mc_hadron['Pb'].query(query)['mc_Pt2'],x_bins=x_bins,fit_reg=None)

   
plt.ylim([0.0,1.3])
plt.tight_layout()
plt.savefig('Broadening.png')
plt.legend()
plt.show()

In [ ]:
print(mc_hadron.keys())

## Calculating broadening

In [ ]:
Edges = np.linspace(0,1.0,10)
bins = [(x, y) for (x, y) in zip(Edges[:-1], Edges[1:])]

y = {}
y['D'] = []
y['Pb'] = []

y_uncorr = {}
y_uncorr['D'] = []
y_uncorr['Pb'] = []

y_unfolded = {}
y_unfolded['D'] = []
y_unfolded['Pb'] = []


x = []

## Calculate the D 
for j, ibin in enumerate(bins):
    query = 'mc_z>%2.2f and mc_z<%2.2f and mc_pid==211 and pass_truth==1'%(ibin[0],ibin[1])
    mean_pt2 = np.average(mc_hadron['D'].query(query)['mc_Pt2'], weights=mc_hadron['D'].query(query)['weight_truth'])
    
    query = 'z>%2.2f and z<%2.2f and pid==211 and pass_reco==1'%(ibin[0],ibin[1])
    mean_pt2_uncorr = np.average(mc_hadron['D'].query(query)['Pt2'], weights=mc_hadron['D'].query(query)['weight_reco'])
    
    query = 'mc_z>%2.2f and mc_z<%2.2f and mc_pid==211 and pass_reco==1'%(ibin[0],ibin[1])
    mean_pt2_unfolded = np.average(mc_hadron['D'].query(query)['mc_Pt2'], weights=mc_hadron['D'].query(query)['weight_reco'])

    
    
    print(mean_pt2)
    
    x.append(0.5*(ibin[0]+ibin[1]))
    
    y['D'].append(mean_pt2)
    y_uncorr['D'].append(mean_pt2_uncorr)
    y_unfolded['D'].append(mean_pt2_unfolded)
    
print (y['D'])

In [ ]:
fig = plt.figure(figsize=(8,6))
plt.plot(x,y_uncorr['D'],'o-',label='D, raw')
plt.plot(x,y_unfolded['D'],'o-',label='D, unfold.')
plt.plot(x,y['D'],'o-',label = 'D, unfold. + acc.')

plt.legend(loc='best',fontsize=18)
plt.xlabel('z')
plt.ylabel('$<p_{T}^{2}>$ [GeV$^{2}$]')

In [ ]:

## Calculate the D 
for j, ibin in enumerate(bins):
    query = 'mc_Zh>%2.2f and mc_Zh<%2.2f and mc_pid==211 and pass_truth==1'%(ibin[0],ibin[1])
    mean_pt2 = np.average(mc_hadron['Pb'].query(query)['mc_Pt2'], weights=mc_hadron['Pb'].query(query)['weight_truth'])
    query = 'Zh>%2.2f and Zh<%2.2f and pid==211 and pass_reco==1'%(ibin[0],ibin[1])
    mean_pt2_uncorr = np.average(mc_hadron['Pb'].query(query)['Pt2'], weights=mc_hadron['Pb'].query(query)['weight_reco'])
    print(mean_pt2)
    
    y['Pb'].append(mean_pt2)
    y_uncorr['Pb'].append(mean_pt2_uncorr)
    
    
print (y['Pb'])

In [ ]:
fig = plt.figure(figsize=(8,6))
plt.plot(x,y['D'],'o-',label = 'D, corrected')
plt.plot(x,y_uncorr['D'],'o-',label='D, uncorrected')

plt.plot(x,y['Pb'],'o-',label = 'Pb, corrected')
plt.plot(x,y_uncorr['Pb'],'o-',label='Pb, uncorrected')

plt.legend(loc='best')
plt.xlabel('z')
plt.ylabel('$<p_{T}^{2}>$ [GeV$^{2}$]')

In [ ]:
plt.plot(x,np.subtract(y_uncorr['Pb'],y_uncorr['D']),'o-',label='Pb, uncorrected')


In [ ]:
fig, axs = plt.subplots(1, 5, sharex=True, sharey=True,figsize=(20,4), gridspec_kw={'hspace': 0, 'wspace':0})


Edges = np.linspace(0,0.6,6)
bins = [(x, y) for (x, y) in zip(Edges[:-1], Edges[1:])]

y = []
y_uncorr = []
x = []
for j, ibin in enumerate(bins):
    print (ibin)
  
    #query ='mc_z>%2.2f and mc_z<%2.2f and mc_pid==211 and pass_truth==1'%(ibin[0],ibin[1])
    #axs[j].hist(mc_hadron['D'].query(query)['mc_PhiPQ'], weights=mc_hadron['D'].query(query)['weight_truth'],bins=10,range=(-180,180),label='Truth',alpha=0.5)
    
    
    query = 'z>%2.2f and z<%2.2f and pid==211 and pass_reco==1'%(ibin[0],ibin[1])
    axs[j].hist(mc_hadron['D'].query(query)['PhiPQ'], weights=mc_hadron['D'].query(query)['weight_reco'],bins=10,range=(-180,180),label='RECO',alpha=0.5)
    
    query = 'z>%2.2f and z<%2.2f and pid==211 and pass_reco==1'%(ibin[0],ibin[1])
    axs[j].hist(mc_hadron['D'].query(query)['mc_PhiPQ'],weights=mc_hadron['D'].query(query)['weight_reco'],range=(-180,180),bins=10,label='C',alpha=0.5)


    #plt.ylim([0.003,0.005])
    
plt.legend(loc='best')

In [ ]:
fig, axs = plt.subplots(1, 5, sharex=True, sharey=True,figsize=(20,4), gridspec_kw={'hspace': 0, 'wspace':0})


Edges = np.linspace(0,0.6,6)
bins = [(x, y) for (x, y) in zip(Edges[:-1], Edges[1:])]

y = []
y_uncorr = []
x = []
for j, ibin in enumerate(bins):
    print (ibin)
  
    query ='mc_z>%2.2f and mc_z<%2.2f and mc_pid==211 and pass_truth==1'%(ibin[0],ibin[1])
    axs[j].hist(mc_hadron['D'].query(query)['mc_Pt2'], weights=mc_hadron['D'].query(query)['weight_truth'],bins=40,range=(0.0,2.0),label='Unfolded +acc.',alpha=0.5)
    
    query = 'z>%2.2f and z<%2.2f and pid==211 and pass_reco==1'%(ibin[0],ibin[1])
    axs[j].hist(mc_hadron['D'].query(query)['Pt2'], weights=mc_hadron['D'].query(query)['weight_reco'],bins=40,range=(0.0,2.0), label='RAW',alpha=0.5)
  
    query = 'z>%2.2f and z<%2.2f and pid==211 and pass_reco==1'%(ibin[0],ibin[1])
    axs[j].hist(mc_hadron['D'].query(query)['mc_Pt2'], weights=mc_hadron['D'].query(query)['weight_reco'],bins=40,range=(0.0,2.0), label='Unfolded',alpha=0.5)

    #query ='mc_z>%2.2f and mc_z<%2.2f and mc_pid==211 and pass_truth==1'%(ibin[0],ibin[1])
    #axs[j].hist(mc_hadron['D'].query(query)['mc_Pt2'],range=(0.0,2.0),**plot_style_2,bins=100)

plt.legend(loc='best')
plt.yscale('log')

In [ ]:
fig, axs = plt.subplots(1, 5, sharex=True, sharey=True,figsize=(20,4), gridspec_kw={'hspace': 0, 'wspace':0})


Edges = np.linspace(0,0.6,6)
bins = [(x, y) for (x, y) in zip(Edges[:-1], Edges[1:])]

y = []
y_uncorr = []
x = []
for j, ibin in enumerate(bins):
    print (ibin)
    query = 'mc_z>%2.2f and mc_z<%2.2f and mc_pid==211 and pass_truth==1'%(ibin[0],ibin[1])
    axs[j].hist(mc_hadron.query(query)['mc_PhiPQ'], weights=mc_hadron.query(query)['weight_truth'],range=(-120,120),**plot_style_1)
    query = 'z>%2.2f and z<%2.2f and pid==211 and pass_reco==1'%(ibin[0],ibin[1])
    axs[j].hist(mc_hadron.query(query)['PhiPQ'], weights=mc_hadron.query(query)['weight_reco'],range=(-120,120),**plot_style_2)


In [ ]:
kinematic = 'mc_Pt2>0.4 and mc_Pt2<0.6'
kinematic += ' and mc_Q2>1.8 and mc_Q2<2.2'
kinematic += ' and mc_Xb>0.2 and mc_Xb<0.3'
query = kinematic + ' and ' + 'mc_z>%2.2f and mc_z<%2.2f and mc_pid==211 and pass_truth==1'%(0.16,0.20)
plt.hist(mc_hadron.query(query)['mc_PhiPQ'], weights=mc_hadron.query(query)['weight_truth'],range=(-180,180),**plot_style_1)
  